# Scraping Tweets from a public Twitter account between two dates

snscraper has a very easy to use python wrapper than can be used to scrape tweets from a public Twitter account.

In [1]:
from tqdm import tqdm
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta
from scrape_utils import scrape_twitter, scrape_twitter_users

In [ ]:
scrape_twitter(query="tropical hut", since="2022-06-01", until="2022-06-15")

In [4]:
df = pd.read_csv("tweets_tropical hut.csv")
df.head()

,tweet_id,date,user_name,tweet,retweets,likes,quote_tweets,replies,retweeted_tweet,quoted_tweet,user_location,user_created
0,1536858402839617536,2022-06-14 23:49:31+00:00,TeamMom_2020,@oslecjunior I also love Tropical Hut burgers...,0,0,0,0,NaN,NaN,philippines,2009-06-11 14:34:37+00:00
1,1536857672481673216,2022-06-14 23:46:37+00:00,yellowbelle,Why didn't I see the tweet in my feed? I grew ...,0,0,0,0,NaN,NaN,🇵🇭,2018-03-15 15:04:28+00:00
2,1536854295643291648,2022-06-14 23:33:12+00:00,rchcalimlim,@agot_isidro Everytime mapunta kami ni wife sa...,0,0,0,0,NaN,NaN,NaN,2009-08-26 22:42:33+00:00
3,1536853561786908672,2022-06-14 23:30:17+00:00,Nicocoleeta,"@greencross123 @joemarireyes I doubt lol, kung...",0,17,1,0,NaN,NaN,NaN,2009-09-19 12:21:51+00:00
4,1536852642345123840,2022-06-14 23:26:37+00:00,sompoqt,"I remember, kapag kukuha ako ng CBC sa Hi-Prec...",0,1,0,0,NaN,https://twitter.com/oslecjunior/status/1536559...,Casa Marasigan,2021-11-18 11:26:30+00:00


In [ ]:
users = df.user_name.unique().tolist()
scrape_twitter_users(users, filename="tropical hut")

In [8]:
df_users = pd.read_csv("tweets_tropical hut_users.csv")
df_users.head(10)

,user_id,user_name,display_name,description,verified,created,followers,following,tweets,location
0,46398861,TeamMom_2020,millennial mom,"huwag bibitaw, revisit assess & consolidate🌸🎀...",False,2009-06-11 14:34:37+00:00,92,355,5055,philippines
1,974300300264710145,yellowbelle,Shine ✨,"came for csi and sherlock, stayed for k-drama ...",False,2018-03-15 15:04:28+00:00,400,259,18558,🇵🇭
2,69120051,rchcalimlim,rch,NaN,False,2009-08-26 22:42:33+00:00,457,1273,4565,NaN
3,75537810,Nicocoleeta,Purifica Octavia,☆☆☆ chicken nuggets ☆☆☆\n\nI didn't vote for t...,False,2009-09-19 12:21:51+00:00,296,263,39045,NaN
4,1461294712912891914,sompoqt,Jonás,"Katoliko, Lider, Anak, Estudyante, Maniniyot",False,2021-11-18 11:26:30+00:00,157,73,1276,Casa Marasigan
5,936873080,LeonZupal,Milano Pilosopo,Padayon! The Proletarian Cause! Baligtarin ang...,False,2012-11-09 13:11:10+00:00,164,705,1885,Winnipeg Canada
6,2284302528,ljernielleyyy,mewo pie ✨,"anxious, gay, tiny | she/her/tita | check my ♡...",False,2014-01-10 00:29:16+00:00,207,208,14407,🏳️‍🌈
7,142598518,unofficialmykr,"michael, as foretold",in transit,False,2010-05-11 08:11:34+00:00,463,702,47092,they/he
8,41079575,NaeSoneul_Jabba,ᴮᴱ Jabba ⁷,Travel. Food. Books. BTS 💜\n🇵🇭,False,2009-05-19 08:25:03+00:00,388,728,5001,"Manila, Philippines"
9,1339836719558918145,3eli3ah3,#RejectMarcosDuterte,they hated 3eli3ah3 for speaking the truth,False,2020-12-18 07:36:01+00:00,15,30,3171,NaN
